In [ ]:
from typing import Callable, List
import pandas as pd
from collections import Counter
from pprint import pprint
from basesqlv3 import BaseSQL
import datetime
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity, paired_distances
from scipy.spatial.distance import cosine
from tqdm import trange
import multiprocessing as mp
import time
from utils import func_timer
from pandarallel import pandarallel
import swifter

pandarallel.initialize(progress_bar=True)

idx = pd.IndexSlice
URI = r"D:\Data\stock_concept.csv"

In [ ]:
conn = dict(host='10.224.16.81', user='haquant', passwd='haquant', database='jydb', port=3306, charset='utf8')
bsql = BaseSQL(conn, read_only=True)
with bsql.start():
    bsql.set_active_table('dailydata')
    cp = bsql.read(field_names=['adjclose'], date_range=dict(start='2017-01-01', end='2022-08-15'))
ret = cp.groupby('wind_code').pct_change()

In [ ]:
stock_concept = pd.read_csv(URI, encoding='GB18030').rename(columns={'DATETIME': 'tradedate'})
stock_concept['tradedate'] = pd.to_datetime(stock_concept['tradedate']).dt.date
stock_concept.set_index(['tradedate', 'wind_code'], inplace=True)
fac = stock_concept['CONCEPT'].astype(str).apply(lambda x: x.split(';')).to_frame('fac')

In [ ]:
def _corr(x: pd.DataFrame) -> pd.DataFrame:
    # x = fac_data.xs(datetime.date(2022, 7, 28))
    import numpy as np
    import pandas as pd
    ix = x['fac'].values
    corr_ = np.array([[len(set(i) & set(j)) for i in ix] for j in ix])
    corr_[np.eye(corr_.shape[0], dtype=np.bool_)] = 0
    corr_ = corr_ / corr_.sum(axis=1)
    corr_[np.eye(corr_.shape[0], dtype=np.bool_)] = -1
    return pd.DataFrame(corr_, index=x.index, columns=x.index)

In [ ]:
data = fac.groupby('tradedate', group_keys=False).parallel_apply(_corr)

In [ ]:
fac.one_hot()